# Introduction to Numerical Modelling for Water Systems

In this lab you will write a simple model for how water infiltrates a porous soil column to replenish soil water storage. These exercises are intentionally simple, aiming to build up your conceptual understanding of how mathematical models can be developed and applied to environmental systems. The principles and concepts you will learn underlie most numerical models of water systems and are thus broadly applicable, rather than specific to the idealised system you will simulate in this lab.

The aims of this lab are to:
* Understand how a numerical model is constructed 
* Implement mathematical equations to describe how stored water volume changes through time
* Understand the importance of initial and boundary conditions in mathematical modelling
* Create and explain graphs/figures that visualise model behaviour
* Experiment with the model to understand its sensitivity to different parameters
* Understand the concepts of transient evolution and steady-state dynamic equilibrium
* Explore the concepts of model calibration and verification

### Python
The model is written in the Python computer programming language. But don't worry, you won't need any prior knowledge of computer programming. We have set up this lab so it will also serve as a basic introduction to programming. 

Learning to program is not the main aim but we want you to see what's "under the hood" of simple models to help you understand how they are built and what they can and cannot do! However, scientific computing is becoming more common place in research and consultancy, so it won't do you any harm to see it in action. Python is highly versatile, for example it can interface with GIS to automate workflows, or it is used in many data analysis and machine learning applications.

**To run a code cell, click in a cell, hold down shift, and press enter.** An asterisk in square brackets `In [*]:` will appear while the code is being executed, and this will change to a number `In [1]:` when the code is finished. *The order in which you execute code cells matters, they must be run in sequence. If the code fails, try restarting and executing all cells again from the top.* To restart the script click the circular arrow button or select `Kernel / Restart` from the menu bar at the top of the window.

Inside blocks of python code each line is a command which will be executed in sequence from top to bottom. Once a variable is created in a code cell it remains active and can continue to be used in further code cells below. There are comments indicated by lines that start with `#`. These lines are not computer code but rather provide commentary and information about what the code is doing to help you follow along. 

**Good commenting is one of the most important aspects of computer programming!** If you modify code or write some from scratch, always document what each line of code is meant to do with a concise comment!

Python has many tools and we don't need to use all of them (it would take forever to load) so we have to tell python which modules we want to use:

In [ ]:
# import some modules that we will need for numerical calculations and for plotting
# we'll shorten their names using "as" so that we don't need to type much later on

# import modules for numerical calculations and for plotting
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os

# tell python to allow plotting to occur within the page
%matplotlib inline

# Customise figure style to use font size 16
from matplotlib import rcParams
rcParams['font.size'] = 16

# import colormaps so we can use shaded values as a function of time
from matplotlib import cm

<img style="float: right; width: 500px; margin: 0px" src="images/2Dsketch.png" alt="Drawing" />

## 3.0 2-D soil water infiltration model

Today we will extend the 1-D column model of soil water infiltration from the previous Lab to two dimensions. The model will represent a vertical cross-section through the soil profile down a hillslope. The model domain extends from the surface exposed to water influx by precipitation to the base where water outflux into an aquifer below occurs. The model will follow a modified version of Richards Equation, which in addition to vertical segregation of water into unsaturated soil will also comprise downslope water percolation due to a topographic gradient in hydraulic head. To keep things simple we will assume that the topographic slope has a uniform angle, $\phi$, from the horizontal and that the model domain is positioned such that the horizontal coordinate points downhill along the surface while the vertical coordinate points downwards into the soil column perpendicular to the surface (see conceptual model sketch).

To discretise the model we divide up the soil cross-section into discrete volume elements. The model will track the water content in each of these grid cells and compute the fluxes of water that flow across the faces between them. We will define the boundary conditions such that water is added along the top boundary by precipition, that unhindered flux of water from the base represents groundwater recharge into a bedrock aquifer beneath, and that unhindered in- and outflow on the side boundaries represent flow from upslope and drainage into downslope portions of the hillslope profile. We assume there is no lateral flow of water in or out of the soil column in the third dimension, hence representing a laterally homogeneous layer.

We will formulate the model in terms of the relative volume fraction, $\theta = V_\mathrm{water}/V_\mathrm{total}$. Based on the fundamental principle of mass conservation, we can express the governing equation for the evolving water content as,

\begin{equation}
    {{\partial \theta(\mathbf{x},t)}\over{\partial t}}=-\boldsymbol{\nabla}{\mathbf{q}(\mathbf{x},t)} \ , \tag{1} \label{eq:1}
\end{equation}

where the change in volume fraction of water in each volume element $\theta(\mathbf{x},t)$ $[L^3/L^3 = 1]$ through time $t$ $[T]$ is given by the spatial gradient in volumetric water flux $\mathbf{q}(\mathbf{x},t) = [q_x,q_z]$ $[L^3/L^2/T = L/T]$ in both spatial directions, $\mathbf{x} = [x,z]$ $[L]$. The gradient operator is a mathematical shorthand for $\boldsymbol{\nabla} = [\partial /\partial x, \partial/\partial z]$, the partial derivative in both spatial directions. Note that we are using boldface symbols to indicate variables that are vectors, meaning they have more than one component to account for all spatial dimensions in the model.

Equation \eqref{eq:1} states that if there is a change in the flux of water in space, $\boldsymbol{\nabla} q$, this has to be balanced by a change in soil water content $\theta$ in time. If more water flows into a volume element than flows out within a given time interval the stored amount in the local volume will increase, and vice versa.

For saturated soil, Darcy's law can be used to describe the water flux $q$ $[L/T]$ as the product of the hydraulic conductivity $K$ $[L/T]$ and the spatial gradient of the hydraulic head $H(\mathbf{x},t)$ $[L/L = 1]$,

\begin{equation}
    \mathbf{q} = -K \boldsymbol{\nabla} H \ . \tag{2} \label{eq:2}
\end{equation}

If the hydraulic head only changes with the topographic slope then we can simplify $\boldsymbol{\nabla} H = [-\sin(\varphi),0]$. Note that $\varphi$ is in units of $[radians]$ here, and the negative sign comes from the hydraulic head _decreasing_ in the x-coordinate direction. In this case, the z-component of the hydraulic gradient is zero since the soil is assumed to already be saturated.

In unsaturated soil, however, we have seen in the previous Lab that the flow of water is governed by two pressure heads: the buoyancy difference between water and air in the unsaturated pore space, which leads to the denser water percolating down to replace air in the pore space, and capillary effects, which act to suck water into unsaturated pore space to wet soil particles. The latter we again represent by an empirically determined function $h(\theta)$ (units are $[L]$, as it is also normalised by $\Delta \rho g$). 

In our 2-D model, buoyancy-related hydraulic head no longer acts purely in the vertical direction. This is due to our choice to point the model's x-axis down along the hillslope and the z-axis into the soil column perpendicular to the surface (see Figure above). The total hydraulic head then is a combination of capillary head and buoyancy-related head modified by hillslope topography, $H = h - \sin(\varphi) x - \cos(\varphi) z$. Note that negative signs again indicate that the head decreases in both coordinate directions. With that, we can use equation \eqref{eq:2} to write the flux components in x- and z-directions as,

\begin{align}
    q_x &= -K \left({{\partial h}\over{\partial x}}\right) - \sin(\varphi)  \ , \tag{3} \label{eq:3} \\
    q_z &= -K \left({{\partial h}\over{\partial z}}\right) - \cos(\varphi)  \ , \tag{4} \label{eq:4}
\end{align}

where we have used that ${\partial x}/{\partial x} = 1$, ${\partial z}/{\partial z} = 1$, and ${\partial x}/{\partial z} = {\partial z}/{\partial x} = 0$.

Substituting eqs (3) and (4) into the conservation equation (1), and noting that both the hydraulic conductivity $K$ and the capillary head $h$ are themselves empirical functions of the soil water content $\theta$, we finally write the governing equation for 2D soil water infiltration,

\begin{equation}
    {{\partial \theta}\over{\partial t}} = -{{\partial }\over{\partial x}} \left[-K(\theta) \left( {{\partial h(\theta)}\over{\partial x}} - \sin(\varphi) \right) \right] 
    -{{\partial }\over{\partial z}} \left[-K(\theta) \left( {{\partial h(\theta)}\over{\partial z}} - \cos(\varphi) \right) \right] \ . \tag{5} \label{eq:5}
\end{equation}

For the variation of the capillary pressure head and the hydraulic conductivity as functions of water content we use the relationships from *van Genuchten* (1980), here shown again as a reminder,

\begin{equation}
h(\Theta) = -{{1}\over{\alpha}} \left(\dfrac{1}{\Theta^{1/m}}-1 \right)^{1/n} \ , \tag{6} \label{eq:6}
\end{equation}
\begin{equation}
    K(\Theta) = K_s \Theta^{1/2}\:\left[1-(1-\Theta^{1/m})^m\right]^2 \ , \tag{7} \label{eq:7}
\end{equation}

with $\Theta$ the relative water content between the residual water content, $\theta_r$, and the saturated water content, $\theta_s$,

\begin{equation}
    \Theta ={{\theta-\theta_r}\over{\theta_s-\theta_r}} \ . \tag{8} \label{eq:8}
\end{equation}


<img style="float: right; width: 500px; margin: 0px" src="images/stencil2D.png" alt="Drawing" />

## 3.1 Numerical Implementation

We discretise the governing equation (1) by dividing the spatial coordinates into $nz \times nx$ volume elements with discrete centre node coordinates $z^i$, $i=1,2,...,nz$, and $x^j$, $j=1,2,...,nx$, spaced at constant step sizes, $\Delta z$, and $\Delta x$, respectively. Time we discretise into $m$ time steps $t^k$, $k=1,2,...,m$, spaced at constant step size $\Delta t$. Using the finite-difference method, we write the discrete form of the governing equation as,

\begin{equation}
    {{\theta^{i,j,k} - \theta^{i,j,k-1}}\over{\Delta t}} = - {{q_z^{i+1/2,j,k-1} - q_z^{i-1/2,j,k-1}}\over{\Delta z}} - {{q_x^{i,j+1/2,k-1} - q_x^{i,j-1/2,k-1}}\over{\Delta x}} \ , \tag{9} \label{eq:9}
\end{equation}

where the discrete flux values into the top ($q^{i-1/2,j}$) and out the bottom ($q^{i+1/2,j}$) of each volume element are given by the relations,

\begin{align}
q_z^{i+1/2,j} = - \dfrac{K^{i+1,j}+K^{i,j}}{2} \left(\dfrac{h^{i+1,j}-h^{i,j}}{\Delta z} - \cos(\varphi) \right) \ , \tag{10} \label{eq:10} \\
q_z^{i-1/2,j} = - \dfrac{K^{i-1,j}+K^{i,j}}{2} \left(\dfrac{h^{i,j}-h^{i-1,j}}{\Delta z} -\cos(\varphi) \right) \ . \tag{11} \label{eq:11} \\
\end{align}

The fluxes into the left and out the right-hand side of each cell can be found by following the same pattern but switching the differences from $i$ to $j$ indices, the grid step size from $\Delta z$ to $\Delta x$, and the buoyancy forcing term from $\cos$ to $\sin$,

\begin{align}
q_x^{i,j+1/2} = - \dfrac{K^{i,j+1}+K^{i,j}}{2} \left(\dfrac{h^{i,j+1}-h^{i,j}}{\Delta x} -\sin(\varphi)\right) \ , \tag{12} \label{eq:12} \\
q_x^{i,j-1/2} = - \dfrac{K^{i,j-1}+K^{i,j}}{2} \left(\dfrac{h^{i,j}-h^{i,j-1}}{\Delta x} -\sin(\varphi)\right) \ . \tag{13} \label{eq:13}
\end{align}

The figure in this cell shows the so-called discretisation stencil, which visualises the spatial relationships and indexing of adjacent discrete cells, and fluxes between them, that are needed to write the discrete governing equation in the cell $i$. You can see that the half indices $j-1/2$ and $j+1/2$ refer to quantities located on the top and base faces, and $i-1/2$ and $i+1/2$ to quantities on the left and right faces of the cell $i,j$, respectively. 

**It is convention to number grid cells in the direction of the coordinate axis.** Here, our z-coordinate axis points downwards from the surface into the soil, and therefore the index numbering goes into the downwards direction as well. The index numbering along the x-axis goes from left to right, which is the downslope direction.

**When using the finite-difference method, we always take the differences from the higher to the lower index.** Always pay close attention to the indexing in your model code, it is one of the most frequent sources of annoying little errors or *bugs* in numerical modelling. Note that we will use the `numpy` function `np.diff()`, which does exactly that: it takes differences between adjacent values in a vector (i.e., a list of numbers), subtracting from each entry the one immediately preceding it in the list.

## 3.2 Model Calibration

In the following exercises we will continue using the empirical relationships between capillary head, $h$, hydraulic conductivity, $K$, and the relative saturation, $\Theta$ from the previous Lab. The model calibration assumes that water content varies between the residual content, $\theta_r$, and the saturated state $\theta_s$. We will use typical values of saturated hydraulic conductivity for silt loam soil and use empirical parameters for the *van Genuchten* (1980) relationships taken from that paper. Note that units are internally consistent, but they are not SI base units!

For later convenience, we also define functions that we can call every time we wish to calculate $\Theta$, $K$, $h$, and also the hydraulic diffusivity, $D$. The latter we will require to appropriately limit the time step for our model.

___INSTRUCTION: Complete these functions in the code cell below. Use the previous Lab's notebook for help if needed.___

In [ ]:
# take soil properties for a silt loam (van Genuchten 1980)
# theta is relative water content cm3/cm3 volume per total volume of soil
# i.e. at saturated porosity theta = 1 and wilting point theta = 0!

theta_s = 0.396           # [cm3/cm3] saturated
theta_r = 0.131           # [cm3/cm3] residual
Ks0     = 4.96            # [cm/day] reference saturated conductivity
alpha   = 4.23*10.**(-3.) # [1/cm] prefactor
n       = 2.06            # [1] powerlaw coefficient
m       = 1.-1./n         # [1] powerlaw coefficient

# define a function to convert absolute water content to relative saturation (scaling between 0 and 1)
def get_THETA(theta):
    Theta = ((theta-theta_r)/(theta_s-theta_r))
    return Theta

# define a function to calculate the hydraulic conductivity as a function of relative saturation
def get_K(theta):
    Theta = get_THETA(theta)
    K = (Theta**0.5)*(1-(1-Theta**(1./m))**m)**2.
    return K

# define a function to calculate the capillary head as a function of relative saturation
def get_h(theta):
    Theta = get_THETA(theta)
    h = -(1./alpha)*(1/Theta**(1/m) - 1.)**(1./n)
    return h

# define a function to calculate the hydraulic diffusivity
def get_D(theta):
    Theta  = get_THETA(theta)
    K      = get_K(theta)
    dh_dth = 1.0*Theta**(-1.0/m)*(-1.0 + Theta**(-1.0/m))**(1.0/n)/(alpha*m*n*(-1.0 + Theta**(-1.0/m))*(theta - theta_r)) # result of symbolic differentiation 
    D      = K*dh_dth # formula for D listed below eq. (13)
    return D

To double check that the empirical relationships have been implemented correctly, let's plot the relationships between soil water content, hydraulic conductivity, and capillary pressure head again as in the previous Lab.

In [ ]:
# define the full range of theta values between residual content and saturation point
theta = np.linspace(theta_r,theta_s,500)[1:-1]

# call the functions for capillary head and hydraulic conductivity
h = get_h(theta)# <== call function to calculate capillary head
K = get_K(theta)# <== call function to calculate hydraulic conductivity

# plot the capillary head
plt.figure(1)
ax = plt.subplot(111)
colour = [0.5,0.5,0.8]
plt.plot(theta, -h, '-',color=colour)
plt.yscale('symlog') # to make y-axis log scale
plt.xlabel('water content [1]')
plt.ylabel('capillary head (–1) [cm]',color=colour)

# colour the left axis
ax.spines['left'].set_color(colour)
ax.spines['top'].set_color(colour)
ax.yaxis.label.set_color(colour)
ax.tick_params(axis='y', colors=colour)

# plot the hydraulic conductivity
plt.twinx()
plt.plot(theta, K, 'k-')
plt.ylabel('rel. hydraulic conductivity [1]');

## 3.3 Numerical Model Setup

Now that we have all we need to program the numerical algorithm to solve the 2D soil water infiltration equation, it's time to prepare the model setup. We will define some parameters that will define our spatial domain size and model timing.


___INSTRUCTION: In the cell below, use the pattern demonstrated to set up the domain depth, vertical grid spacing, and coordinate vector in z-direction to also set up parameters for the domain width, horizontal grid spacing, and horizontal coordinate vector.___

In [ ]:
# Setup model domain and coordinates in two dimensions x, z
depth = 100.     # domain depth [cm]
dz    = 1.       # vertical grid spacing [cm]
z     = np.arange(0.    ,depth      ,dz)  # coordinate vector for grid cell centre nodes
zm    = np.arange(-dz/2.,depth+dz/2.,dz)  # coordinate vector for grid cell face nodes

width = 200. # <== add a parameter for domain width [cm]
dx = 1. # <== add a parameter for horizontal grid step [cm]
x = np.arange(0., width, dx) # <== set up x-coordinate vector for grid cell centre nodes
xm = np.arange(-dx/2., width+dx/2., dx) # <== set up x-coordinate vector for grid cell face   nodes

The next code block creates a function that we will use to plot the model output into formatted figures as the model run proceeds. Don't worry too much about the detail here but do quickly take a look through to see if you can understand how it's done.

In [ ]:
def plot_fig(x,z,field1,field2,field3,runID,frame):

    # Set up the figure and add the axis
    fig = plt.figure(figsize=(8,15))
    ax1 = fig.add_subplot(311)

    # Set the x axis limits, move labels to the top of the box and label
    ax1.set_title("Water content [vol]")
    ax1.set_xlim(0,width)
    ax1.xaxis.tick_top()
    ax1.xaxis.set_label_position('top') 

    # Set the y axis limits and invert the y axis
    ax1.set_ylim(0,depth)
    ax1.invert_yaxis()
    ax1.set_ylabel("Depth [cm]")

    # Create an empty image object that we will populate repeatedly to visualise model results
    im1 = ax1.imshow(field1, extent=[0,width,0,depth], origin='lower')
    
    # Create colorbar
    divider = make_axes_locatable(ax1)
    cax1 = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax1)
    
    
    # Set up the figure and add the axis
    ax2 = fig.add_subplot(312)

    # Set the x axis limits, move labels to the top of the box and label
    ax2.set_title("Vert. flux [cm/day]")
    ax2.set_xlim(0,width)
    ax2.xaxis.tick_top()
    ax2.xaxis.set_label_position('top') 

    # Set the y axis limits and invert the y axis
    ax2.set_ylim(0,depth)
    ax2.invert_yaxis()
    ax2.set_ylabel("Depth [cm]")

    # Create an empty image object that we will populate repeatedly to visualise model results
    im2 = ax2.imshow(field2, extent=[0,width,0,depth], origin='lower')
    
    # Create colorbar
    divider = make_axes_locatable(ax2)
    cax2 = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im2, cax=cax2)
    
    
    # Set up the figure and add the axis
    ax3 = fig.add_subplot(313)

    # Set the x axis limits, move labels to the top of the box and label
    ax3.set_title("Horiz. flux [cm/day]")
    ax3.set_xlim(0,width)
    ax3.xaxis.tick_top()
    ax3.xaxis.set_label_position('top') 

    # Set the y axis limits and invert the y axis
    ax3.set_ylim(0,depth)
    ax3.invert_yaxis()
    ax3.set_ylabel("Depth [cm]")

    # Create an empty image object that we will populate repeatedly to visualise model results
    im3 = ax3.imshow(field3, extent=[0,width,0,depth], origin='lower')

    # Create colorbar
    divider = make_axes_locatable(ax3)
    cax3 = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im3, cax=cax3)
    
    # save the results as an image file
    # rename this for every parameter variation you test
    Outpath = os.getcwd()+runID
    if not os.path.isdir(Outpath):
        os.mkdir(Outpath)
    fig.savefig(Outpath+runID+"_"+str(frame)+".png",dpi=300)
    
    return

<div class="alert alert-block alert-info">
<font color="black">
<h3>Task 1</h3>
<p> In the model code below, locate the line where the soil water content, $\theta$, is updated and fill in the formula you find when reordering the discretised model eq. (9) above into the form of an explicit update. Use the numpy function np.diff() to take the finite difference of the flux components `qx` and `qz`.

**Hint 1:** An explicit update is when a single unknown quantity appears to the left, and all known quantities to the right of the equal sign.
    
**Hint 2:** Check how the flux components `qx` and `qz` are calculated from the finite difference of the hydraulic head in the provided code below. That should give you an idea how to translate eq. (9) to code instructions.
    
Once you've completed the code, execute the cell to run the model and see if you can understand what the results show.
<p></p>
</font>
</div>

Two more things to note before you get stuck in with building the 2-D model:

We are placing the main model routine into a function definition so we can later call it without copy-pasting too much code. You should by now be familiar with this procedure.

The initial setup of the arrays to store the water content and water fluxes are set up the same way as in the previous lab, except that they are now 2-dimensional data matrices rather than 1-D data lists. Note that Python follows the indexing convention that vertical indices come before horizontal. For example, to call the point i = 20, j = 10 from the array of water contents, theta, you would write `theta[10,20]`.

In [ ]:
# prepare function to contain main model routine
def infiltration2D(runID,phi,Pe,Ks,tend,dPlotTime):

    ################################
    #####  MAIN MODEL ROUTINE  #####
    ################################

    # set initial condition for theta to be ever so slightly above the residual water content
    theta = np.zeros((len(z),len(x))) + theta_r + 1e-3

    # initialise a vector to calculate flux values on cell faces
    qx = np.zeros((len(z)+0,len(x)+1))
    qz = np.zeros((len(z)+1,len(x)+0))

    # reset timing parameters
    t         = 0.     # starting time [days]
    PlotTime  = 0.     # initialise plot time count

    while t < tend:

        # get capillary and total pressure head
        h = get_h(theta)

        # get hydraulic conductivity
        K = Ks * get_K(theta)

        # interpolate conductivity values to cell faces
        Kmz = (K[1:,:] + K[:-1,:])/2
        Kmx = (K[:,1:] + K[:,:-1])/2

        # get water flux on cell faces
        qz[1:-1,:] = - Kmz * (np.diff(h, axis=0)/dz - np.cos(phi*np.pi/180))
        qx[:,1:-1] = - Kmx * (np.diff(h, axis=1)/dx - np.sin(phi*np.pi/180))

        # set constant in/out-flux boundary conditions on top and base of domain
        qz[ 0,:] = Pe;        # top influx boundary is precipitation
        qz[-1,:] = qz[-2,:];  # base outflux boundary free outflux (dq/dz = 0)
        qx[:, 0] = qx[:, 1];  # left boundary free influx (dq/dx = 0)
        qx[:,-1] = qx[:,-2];  # right boundary free outflux (dq/dx = 0)

        # get hydraulic conductivity to limit time step
        D = Ks * get_D(theta)

        # limit stable time step for diffusive and advective transport
        dt = np.min([dz**2/4/np.max(D), dz/4/np.max(abs(qz))])/2

        # update theta with the flux gradient
        theta -= (np.diff(qz,axis=0)+np.diff(qx,axis=1))*dt #<== fill in formula to update water content following eq. (9)

        # limit theta to remain between residual and saturation points
        theta = np.clip(theta,theta_r+1e-6,theta_s-1e-6)

        # update Time
        t += dt        

        if t > PlotTime:
            # print model diagnostics
            print('    ---  dt = %4.4e;  t = %4.4e;  max qx = %4.4e;  max qz = %4.4e;  max theta = %4.4e;' % (dt,t,np.max(qx[1:-1,1:-1]),np.max(qz[1:-1,1:-1]),np.max(theta)))
            # plot evolving water content and fluxes
            plot_fig(x,z,theta,qz,qx,runID,round(PlotTime/dPlotTime))
            plt.show()
            PlotTime += dPlotTime
            
    return

<div class="alert alert-block alert-info">
<font color="black">
<h3>Task 2</h3>
<p> Now that you've completed the function, use the next cell to call that function and run the model. See if you can understand what the results show. How does it compare to the 1-D model from the previous Lab?
<p></p>
</font>
</div>

Here's a few more bits of information regarding the input parameters you can set to control the model.

* At the top of the cell you will find a parameter `runID`, which you can set to any string of characters. This will be used to label the output figures the code will automatically print into a subfolder `out/runID/` located in the same folder where you iPython notebook is opened from. Make sure to reset this tag every time you run the model with different parameters so that figures from one run are not overwritten with those of a next one.

* A parameter `phi` is prepared near the top of the model routine which you can use to set the angle of the hillslope in $[^\circ]$. Note that, as a consequence, we will need to transform the angle from degrees to radians once plugged into sine or cosine functions.

* As in the previous lab you will be able to set the effective precipitation rate, `Pe` [cm/day], at which water is being delivered to the top of the soil column. 

* `Ks` is the saturated hydraulic conducitivity of the soil, for which we have set a calibrated value above. For now, just use that set value, `Ks0`. In a later task will do some more interesting things with it.

* Timing parameters are prepared for you to control how long the model will run for and how often the model will plot output and save figures. Note that we will use the stable time step limiter introduced in the 1-D model, so there is no need to set a time step size ahead of running the model.

* Note that the model function does not return any output. That is ok since the function itself plots output figures and prints them to file for you.

In [ ]:
# set run identifier tag (used to name output figures)
runID = "demo"  # set new ID tag for every model run!

# set hillslope angle [degrees]
phi = 10

# set the top boundary condition
Pe  = 5 # preciptitation rate in [cm/day]

# set the saturated conductivity
Ks  = Ks0 # preciptitation rate in [cm/day]

# set model timing parameters
tend      = 2.     # stopping time [days]
dPlotTime = 2./24. # plotting time interval (every 2 hours)

# run model with set parameters 
infiltration2D(runID,phi,Pe,Ks,tend,dPlotTime) # <== call function infiltration2D() using the parameters set above as input arguments

## 3.4 Lateral variability

In your initial 2-D model you will have seen how water is infiltrating the soil layer in the vertical direction as well as percolating downhill at a somewhat lower speed depending on the hillslope angle you set. However, the model behaviour isn't really two-dimensional since there isn't yet any variability of model behaviour in the along-slope direction. For the present model scenario, spending the additional conceptual and computational effort is not worth it because the same outcome could be captured by a less conceptually complex and computationally costly 1-D model.

Extending a model to two dimensions is justified if some aspect of the model introduces lateral variability. In the following, we will build several versions of the model that exhibit such laterally inhomogeneous 2-D behaviour.

A first possible cause of lateral variability is variable water influx from the top. For example, it is common that precipitation is more intense on the windward upper slopes of a ridge of hills, a phenomenon known as orographic precipitation. On the relatively small spatial scales on which our model is built such effects will likely be negligible in natural systems, but for the sake of demonstration we will consider a scenario where precipitation falls predominantly on the uphill portion of the model domain.

<div class="alert alert-block alert-info">
<font color="black">
<h3>Task 3</h3>
<p> Create a model where precipitation is concentrated to the upslope portion of the model domain. We will use a Gaussian bell-curve distribution with the peak located on the left boundary at $x=0$ and with an amplitude of $Pe_0$ decaying away towards zero over a third the width, $W/3$, of the model domain.
    
The formula to use to describe the precipitation rate as a function of the x-coordinate is,
$$Pe(x) = Pe_0 \exp\left( {{-x^2} \over {(W/3)^2}} \right)  $$

Translate this formula into Python code and set up the laterally variable precipitation rate `Pe` at the start of the model. Then replace `Pe0` by `Pe` where the top boundary condition is applied.
    
___How changes do you observe in the model behaviour?___
<p></p>
</font>
</div>

In [ ]:
# set run identifier tag (used to name output figures)
runID = "upslope_prec"

# set hillslope angle (degrees)
phi = 10

# set the top boundary condition
Pe0 = 5 # preciptitation rate in [cm/day]
Pe  = # <== fill in formula for precipitation concentrated to upslope area

# set the saturated conductivity
Ks  = Ks0 # preciptitation rate in [cm/day]

# set model timing parameters
tend      = 2.     # stopping time [days]
dPlotTime = 2./24. # plotting time interval (every 2 hours)

# run model with set parameters 
# <== call function infiltration2D() using the parameters set above as input arguments

## 3.5 Heterogeneous soil profile

Another possible reason for lateral variability that would require a 2-D modelling approach is if the soil profile has spatially heterogenous properties that may introduce additional variability in soil water infiltration. The empirical model we use to describe hydraulic conductivity and capillary effects contains a number of calibrated parameters that can vary between different soil types. 

For simplicity, we will focus on the saturated conductivity, $K_s$ here. So far, we have assumed that this parameter is constant and uniform in our model. However, in natural soils, this parameter will generally vary in space (and likely also time) as a function of a number of properties including grain size- and shape-distribution, soil composition, and many more.

Here we will test a combination of two effects: a smooth random perturbation to saturated conductivity, and a horizontal layer of increased conductivity within the soil layer.

<div class="alert alert-block alert-info">
<font color="black">
<h3>Task 4</h3>
<p> Create a model with heterogeneous hydraulic conductivity following these steps:
    
* Set up a 2-D matrix for the saturated conductivity `Ks` to replace the previously uniform parameter `Ks0`. To get started, set the entire matrix to the value of `Ks0`. You can use the numpy function `np.zeros(np.shape(theta))` to create a matrix of the right dimensions
    
* In a layer extending from 1/3 to 2/3 of the model depth and running along the entire width of the domain, set the saturated conductivity to `10*Ks0`. Use logical indexing to select all nodes within a distance smaller than `depth/6` of the z-coordinate located at `depth/2`: `np.abs(z-depth/2) < depth/6`

* Use the code provided to add smoothed random noise to the hydraulic conductivity field. You can set two parameters, the (integer) smoothness number `smth > 1`, and the perturbation factor, `pert > 1` to scale up or down the amplitude of random perturbation added.
    
* Now that the spatially heterogeneous `Ks` is set up, copy down the model routine from before and run the model to observe the consequences.

<p></p>
</font>
</div>

In [ ]:
# set run identifier tag (used to name output figures)
runID = "upslope_prec"

# set hillslope angle (degrees)
phi = 10

# set the top boundary condition
Pe0 = 5 # preciptitation rate in [cm/day]
Pe  = Pe0 * np.exp(-x**2/(width/3)**2) # precipitation concetrated to upslope portion

# set saturated conductivity with random noise and increased conductivity layer
Ks0   = 5.   # reference saturated conductivity [cm/day]
pert  = 10.  # amplitude of noise to be added
smth  = 10   # level of smoothing of noise (integer smth>=1)

Ks = np.zeros((len(z),len(x)))   # <== set up 2D-array and set all points equal to Ks0
Ks[...] = 10*Ks[...] # <== use logical indexing to increase conductivity of middle third layer

noise = np.random.random((len(z),len(x)))-0.5  # use random numbers to initialise noise
Ks    = Ks * pert**noise # add noise to saturated conductivity

for k in range(5):
    Ks[1:-1,1:-1] = Ks[1:-1,1:-1] + np.diff(Ks[:,1:-1],n=2,axis=0)/8 \
                                  + np.diff(Ks[1:-1,:],n=2,axis=1)/8    # use diffusion operator to smooth noise
    Ks[0,:] = Ks[1,:]; Ks[-1,:] = Ks[-2,:]
    Ks[:,0] = Ks[:,1]; Ks[:,-1] = Ks[:,-2]

# set model timing parameters
tend      = 2.     # stopping time [days]
dPlotTime = 2./24. # plotting time interval (every 2 hours)

# run model with set parameters 
# <== call function infiltration2D() using the parameters set above as input arguments

## 3.6 Time-dependent precipitation

In natural systems, precipitation of course isn't constant through time but instead varies between rainfall events and dry spells in between. As a last step for this Lab we will now produce an instance of the model where precipitation varies according to a cosine function with a set amplitude $Pe_0$ and period $\tau_\mathrm{Pe}$,

\begin{equation}
    Pe(t) = \dfrac{Pe_0}{2} \left[1 + \cos\left(\dfrac{2 \pi t}{\tau_\mathrm{Pe}} \right) \right] \ . \tag{14} \label{eq:14}
\end{equation}

Since this function will change when time $t$ is updated it needs to be integrated into the model function itself and cannot simply be sorted out as initial condition. That will be your final task for this Lab.

<div class="alert alert-block alert-info">
<font color="black">
<h3>Task 5</h3>
<p> Modify the model function by adding the formula in eq. (14) in place of the existing top boundary condition.
    
* In the new function definition provided below, copy in the contents of the model function from above. 

* Identify the place where the top boundary condition is set and replace `Pe` by the formula in eq. (14).
    
* Note that the function definition already features two new input parameters, `Pe0` and `tau_Pe`. Use these parameters along with the existing time variable `t` in the new instruction for time-dependent precipitation.
    
* Once the updated function is ready, use it in the code cell below to investigate how water supplied by periodic rainfall infiltrates into the soil layer over time. Test periods of half a day, one day, and two days.

<p></p>
</font>
</div>

In [ ]:
# prepare function to contain main model routine
def infiltration2D_timedep(runID,phi,Pe0,tau_Pe,Ks,tend,dPlotTime):

    # <== fill in model function from above and modify top boundary condition to provide
    #     time-dependent precipitation input
            
    return

In [ ]:
# set run identifier tag (used to name output figures)
runID = "timedep_prec"

# <== fill in all parameters from above including spatially variable precipitation and conductivity

# run model with set parameters 
# <== call function infiltration2D_timedep() using the parameters set above as input arguments
